### Set up

In [ ]:
from pymarketcap import *  
import json
import pprint
import pandas as pd
from datetime import datetime, timedelta 
from bs4 import BeautifulSoup
import urllib
import time
from apscheduler.schedulers.blocking import BlockingScheduler


def setdate(x):
    if len(x) == 5 or len(x) == 7:
        return datetime.today().strftime("%m/%d/%Y")
    elif len(x) == 8:
        newDate = datetime.today() - timedelta(days=int(x[:1]))
        return newDate.strftime("%m/%d/%Y")
    else:
        newDate = datetime.today() - timedelta(days=int(x[:2]))
        return newDate.strftime("%m/%d/%Y")

### Market Stats

In [ ]:
def grab_info():

    moment=time.strftime("%Y-%b-%d__%H_%M_%S",time.localtime())

    coinmarketcap = Pymarketcap()

    #Stats for coinmarketcap
    stats = coinmarketcap.stats(VERBOSE=True)

    #Nothing in Ticker brings in all coin data
    #coinmarketcap.ticker('STEEM')
    #coinmarketcap.ticker('IFT')
    coins = coinmarketcap.ticker(VERBOSE=True)

    #Exchange information for a particular coin
    #Each coin will also variable
    markets = coinmarketcap.markets('STEEM',VERBOSE=True)
    market1coin = 'STEEM'

    #You can pass '7d', '24h', '1h', 'gainers' and 'losers' for filter responses:
    #coinmarketcap.ranks('gainers', '1h', V=True)
    ranks = coinmarketcap.ranks(V=True)

    #Writing all files
    with open('coins'+moment+'.txt', 'w') as outfile:
        json.dump(coins, outfile)
    with open('stats'+moment+'.txt', 'w') as outfile:
        json.dump(stats, outfile)   
    with open('markets'+market1coin+moment+'.txt', 'w') as outfile:
        json.dump(markets, outfile) 
    with open('ranks'+moment+'.txt', 'w') as outfile:
        json.dump(ranks, outfile)  
grab_info()

### New coins

In [ ]:
#Grabs new coins from coinmarket cap and saves as CSV

def new_coin(): 
    r = urllib.urlopen('http://coinmarketcap.com/new/').read()
    soup = BeautifulSoup(r, "lxml")
    col = soup.find('tr').find_all('td')
    for c in col:
        print(c.get_text())
    final_list = []
    tr = soup.findChildren('tr') 
    final_list = []
    for t in tr:  
        td = t.findChildren('td')  
        coin_list = []
        for child in td:
            coin_list.append(child.text.replace('\n', '').strip().replace(" ", ""))
        if coin_list != []:
            final_list.append(coin_list)

    data = pd.DataFrame(final_list, columns = ["Name","Symbol", "Added", "Market Cap","Price","Circulating Supply","Volume (24h)", "% 24h"])        
    data["Added"] = data["Added"].apply(lambda x: setdate(x))
    data.to_csv('new'+moment+'.csv', sep=',')  
new_coin()

### Run block scheduler    

In [ ]:
scheduler = BlockingScheduler()
scheduler.add_job(new_coin, 'interval', hours=1)
scheduler.add_job(grab_info, 'interval', hours=1)
scheduler.start()